# Activity - Friends By Age

Task: Import a text file to compute the average number of friends by age in a scoial network.

In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.19:4040
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1588782016608)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@c308a64


Load each line of the source text file into an RDD.

In [2]:
val lines = sc.textFile("../../data/fakefriends.csv")

lines: org.apache.spark.rdd.RDD[String] = ../../data/fakefriends.csv MapPartitionsRDD[1] at textFile at <console>:25


Print the first couple of lines to see how the data looks like.

In [4]:
lines.top(5).foreach(println)

99,Keiko,69,491
98,Will,44,178
97,Nerys,69,361
96,Ezri,25,233
95,Odo,29,173


Each row represents `index, Name, Age, NumOfFriends`.

We define a helper function `parseLines` to convert the `(age, numFriends)` tuples.

**PROTOTYPING**

In [5]:
val sampleTxt = lines.top(5)

sampleTxt: Array[String] = Array(99,Keiko,69,491, 98,Will,44,178, 97,Nerys,69,361, 96,Ezri,25,233, 95,Odo,29,173)


In [7]:
sampleTxt(0).split(",")

res4: Array[String] = Array(99, Keiko, 69, 491)


In [8]:
def parseLine(line: String) = {
    val fields = line.split(",")       //Split by commas
    val age = fields(2).toInt          //Extract age and convert to Int
    val numFriends = fields(3).toInt   //Extract numFriends and convert to Int
    (age, numFriends)                  // Create a tuple of our result 
}

parseLine: (line: String)(Int, Int)


Use our `parseLines` method to convert.

In [10]:
val rdd = lines.map(parseLine(_))

rdd: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[5] at map at <console>:28


We are starting with an RDD of the form `(age, numFriends)` where `age` is the KEY and `numFriends` is the VALUE.

We use `mapValues` to convert each `numFriends` value to a tuple of `(numFriends, 1)` and then we use `reduceByKey` to sum up the total `numFriends` and total instancces for each age, by adding together all the `numFriends` values and 1s respectively.

In [12]:
val totalByAge = {rdd.mapValues (x => (x, 1))
                     .reduceByKey( (x,y)=> (x._1 + y._1, x._2 + y._2))
                 }

totalByAge: org.apache.spark.rdd.RDD[(Int, (Int, Int))] = ShuffledRDD[7] at reduceByKey at <console>:27


So now we have tuples of `(age, (totalFriends, totalInstances))`. To compute the average we divide `totalFriends` by the `totalInstances` for each age.

In [ ]:
val averagesBy